In [1]:
from pyscf import gto,scf
import numpy as np 
from numpy.linalg import norm as norm
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from pyscf.grad import rhf as grhf
from pyscf.hessian import rhf as hrhf
import basis_set_exchange as bse
from pyscf.geomopt.berny_solver import optimize
from berny import Berny, geomlib,Geometry,Math
import berny.coords
import sys
sys.path.append("/home/administrator/Documenti/APDFT/prototyping/hessian/AAFF/Libs/")
from Morse import mpa
from AP_class import APDFT_perturbator as APP
from aaff import aaff,alc_deriv_grad_nuc,alc_differential_grad_nuc
from FcMole import FcM,FcM_like
from alch_deriv import alch_deriv
from FDcoeffs import *
ang2bohr=1.8897261246
bohr2ang=.5291772109

In [2]:
l=3.3376
x=-(1/24)**.5*l
z=(2/3)**.5*l+x
ch4str="C 0 0 0; H 0 0  {}; H  0  {} {}; H {} {} {}; H {} {} {}".format(z,l/3**.5,x,l/2,-l*3**.5/6,x,-l/2,-l*3**.5/6,x)

In [3]:
def build_h_ic(s,g_ic,h0,B_inv):
    geom0=s.geom.copy()
    B=s.coords.B_matrix(geom0)
    bms=[]
    for i in range(geom0.coords.flatten().shape[0]):
        a=geom0.coords.flatten()
        geom1=geom0.copy()
        a[i]+=.001*bohr2ang
        a=a.reshape(geom0.coords.shape)
        geom1.coords=a
        bms.append((s.coords.B_matrix(geom1)-B)*1000)
    bms_arr=np.asarray(bms)
    BpG2=np.einsum('ijk,j->ik',bms,g_ic)
    h_ic=B_inv.T@(h0-BpG2)@B_inv
    return h_ic
tbbde={"CH":105/627.5,"HC":105/627.5,"NH":110/627.5,"HN":110/627.5,\
       "OH":119/627.5,"HO":119/627.5,"FH":136/627.5,"HF":136/627.5}

In [4]:
def mpa_pb(coords,atoml,g,h,gic=False,solve_ic=False,ghost=[]):
    g=np.asarray(g)
    if not len(h.shape)==2:
        h=h.swapaxes(1,2)
        h=h.reshape(g.shape[0]*3,g.shape[0]*3)
    geom0=Geometry(atoml,coords*bohr2ang)
    bernyobj=Berny(geom0)
    s=bernyobj._state
    B = s.coords.B_matrix(geom0)
    q0=s.coords.eval_geom(geom0)
    B_inv = B.T.dot(Math.pinv(np.dot(B, B.T)))
    if not gic:
        g_ic=np.dot(B_inv.T, (g).reshape(-1))
    else:
        g_ic=g
  #  print (s.coords._coords)
   # for c in range(len(s.coords._coords)):
   #     if s.coords._coords[c].__class__ is berny.coords.Bond:
   #         if s.coords._coords[c].i in ghost or s.coords._coords[c].j in ghost:
   #             g_ic[c]=0
   #     if s.coords._coords[c].__class__ is berny.coords.Angle:
   #         if s.coords._coords[c].i in ghost or s.coords._coords[c].j in ghost or s.coords._coords[c].k in ghost:
   #             g_ic[c]=0
                
    h_ic=build_h_ic(s,g_ic,h,B_inv)
    #print (s.coords._coords)
    if not solve_ic:    
        return mpa(g_ic[0],h_ic[0,0],q0[0],tbbde[atoml[s.coords._coords[0].i]+atoml[s.coords._coords[0].j]])[0]\
                +g_ic[0]/h_ic[0,0]-q0[0]
    bnr=0
    ddq_mb=np.zeros_like(q0)
    for i in range(len(s.coords._coords)):
        if s.coords._coords[i].__class__ is berny.coords.Bond:
            bnr+=1
            if s.coords._coords[i].i not in ghost and s.coords._coords[i].j not in ghost:
                bondatoms=atoml[s.coords._coords[i].i]+atoml[s.coords._coords[i].j]
                ddq_mb[i]+=mpa(g_ic[i],h_ic[i,i],q0[i],tbbde[bondatoms])[0]+g_ic[i]/h_ic[i,i]-q0[i]
    dq_NR=-np.linalg.solve(h_ic,g_ic)
    ddq_mb[bnr:]=np.linalg.solve(h_ic[bnr:,bnr:],-h_ic[bnr:,:]@(ddq_mb))
    return q0,dq_NR,q0+dq_NR,q0+dq_NR+ddq_mb

In [5]:
CH4=gto.M(atom=ch4str,unit="bohr",basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])})
mf = scf.RHF(CH4)
mf.scf()
g0=mf.Gradients().grad()
h0=mf.Hessian().hess()
ch4coords=CH4.atom_coords()

converged SCF energy = -40.2151804034984
--------------- RHF gradients ---------------
         x                y                z
0 C    -0.0000000000     0.0000000000     0.0000000000
1 H    -0.0000000000     0.0000000000    -0.0000007926
2 H    -0.0000000000    -0.0000007473     0.0000002642
3 H    -0.0000006472     0.0000003736     0.0000002642
4 H     0.0000006472     0.0000003736     0.0000002642
----------------------------------------------


In [6]:
afC=aaff(mf,[1,0,0,0,0])+alc_deriv_grad_nuc(CH4,[1,0,0,0,0])
afH1=aaff(mf,[0,0,0,0,-1])+alc_deriv_grad_nuc(CH4,[0,0,0,0,-1])
afH2=aaff(mf,[0,0,0,-1,0])+alc_deriv_grad_nuc(CH4,[0,0,0,-1,0])
afH3=aaff(mf,[0,0,-1,0,0])+alc_deriv_grad_nuc(CH4,[0,0,-1,0,0])

In [7]:
nh3optg,oh2optg,fhoptg=np.load("optgeoms.npy")

In [8]:
#getting inverse of the hessian matrix
H1=np.linalg.inv((np.swapaxes(h0,1,2)).reshape(15,15))
h1=H1.reshape(5,3,5,3)
def invhess(h):
    H=np.linalg.inv((np.swapaxes(h,1,2)).reshape(h.shape[0]*h.shape[2],h.shape[0]*h.shape[2]))
    h1=H.reshape(h.shape[0],h.shape[2],h.shape[1],h.shape[3])
    return h1

In [9]:
nh3predictg=(ch4coords-np.einsum('ijkl,kl->ij',h1,  afC+afH1))[:4]
oh2predictg=(ch4coords-np.einsum('ijkl,kl->ij',h1,  2*afC+afH1+afH2))[:3]
fhpredictg=(ch4coords- np.einsum('ijkl,kl->ij',h1,  3*afC+afH1+afH2+afH3))[:2]

In [10]:
#place non H atom in the origin
nh3predictg-=nh3predictg[0]
oh2predictg-=oh2predictg[0]
fhpredictg-=fhpredictg[0]
nh3optg-=nh3optg[0]
oh2optg-=oh2optg[0]
fhoptg-=fhoptg[0]

In [11]:
#bond lengths
print(norm(ch4coords[1]))
print(norm(nh3predictg[1]),norm(nh3optg[1]))
print(norm(oh2predictg[1]),norm(oh2optg[1]))
print(norm(fhpredictg[1]),norm(fhoptg[1]))

2.043854241378284
1.877102528301789 1.8856061076490165
1.7017428542299706 1.7756053665813991
1.5148039815344685 1.69574139


In [12]:
def anglecenter(coords):
    return np.around(np.arccos(coords[1].dot(coords[2])/norm(coords[1])/norm(coords[2]))*180/np.pi,decimals=3)

In [13]:
#angles  
print(anglecenter(ch4coords))
print(anglecenter(nh3predictg),anglecenter(nh3optg))
print(anglecenter(oh2predictg),anglecenter(oh2optg))

109.471
103.102 108.193
96.677 106.393


In [14]:
minimum=(norm(nh3optg[1]),anglecenter(nh3optg)*np.pi/180)

## Rotations 

In [15]:
rv=np.asarray([0,0,1])*np.pi*2/3
r = R.from_rotvec(rv)

In [16]:
np.set_printoptions(precision=5,suppress=True)

In [17]:
afH2

array([[ 0.02542, -0.01467, -0.01038],
       [ 0.0125 , -0.00722, -0.011  ],
       [ 0.0125 , -0.01278, -0.00314],
       [-0.06774,  0.03911,  0.02766],
       [ 0.01732, -0.00444, -0.00314]])

In [18]:
b=r.apply(afH1)
b[[2,3,4]]=b[[3,4,2]]
b #=afH2

array([[ 0.02542, -0.01467, -0.01038],
       [ 0.0125 , -0.00722, -0.011  ],
       [ 0.0125 , -0.01278, -0.00314],
       [-0.06774,  0.03911,  0.02766],
       [ 0.01732, -0.00444, -0.00314]])

In [19]:
afH3

array([[-0.     ,  0.02935, -0.01038],
       [-0.     ,  0.01444, -0.011  ],
       [ 0.     , -0.07822,  0.02766],
       [-0.00481,  0.01722, -0.00314],
       [ 0.00481,  0.01722, -0.00314]])

In [20]:
c=r.apply(r.apply(afH1))
c[[2,3,4]]=c[[4,2,3]]
c

array([[ 0.     ,  0.02935, -0.01038],
       [-0.     ,  0.01444, -0.011  ],
       [ 0.     , -0.07822,  0.02766],
       [-0.00481,  0.01722, -0.00314],
       [ 0.00481,  0.01722, -0.00314]])

In [21]:
r.apply(r.apply(h0[4,4]).T),h0[3,3]

(array([[ 0.26311, -0.12201, -0.08627],
        [-0.12201,  0.12223,  0.04981],
        [-0.08627,  0.04981,  0.087  ]]),
 array([[ 0.26311, -0.12201, -0.08627],
        [-0.12201,  0.12223,  0.04981],
        [-0.08627,  0.04981,  0.087  ]]))

In [22]:
r.apply(r.apply(h0[3,2]).T).T,h0[2,4]

(array([[ 0.00187, -0.02882,  0.01207],
        [ 0.00065, -0.01439,  0.00506],
        [ 0.00165, -0.01298,  0.0044 ]]),
 array([[ 0.00187, -0.02882,  0.01207],
        [ 0.00065, -0.01439,  0.00506],
        [ 0.00165, -0.01298,  0.0044 ]]))

In [23]:
r.apply(r.apply(h0[3,1]).T).T,h0[2,1]

(array([[-0.00119,  0.     ,  0.     ],
        [ 0.     ,  0.00289, -0.00007],
        [ 0.     ,  0.03602, -0.00982]]),
 array([[-0.00119,  0.     , -0.     ],
        [ 0.     ,  0.00289, -0.00007],
        [ 0.     ,  0.03602, -0.00982]]))

In [24]:
r.apply(r.apply(h0[0,3]).T).T,h0[0,2]

(array([[-0.05433,  0.     ,  0.     ],
        [ 0.     , -0.30766,  0.08956],
        [-0.     ,  0.08956, -0.086  ]]),
 array([[-0.05433, -0.     ,  0.     ],
        [-0.     , -0.30766,  0.08956],
        [-0.     ,  0.08956, -0.086  ]]))

In [25]:
r.apply(r.apply(h0[1,0]).T).T,h0[1,0]

(array([[-0.05433, -0.     , -0.     ],
        [-0.     , -0.05433, -0.     ],
        [-0.     , -0.     , -0.33932]]),
 array([[-0.05433, -0.     , -0.     ],
        [-0.     , -0.05433,  0.     ],
        [-0.     ,  0.     , -0.33932]]))

In [26]:
def rothess(h):
    hr=np.zeros_like(h)
    ridx={0:0,1:1,2:3,3:4,4:2}
    for i in range(5):
        for j in range(5):
            hr[i,j]=r.apply(r.apply(h[ridx[i],ridx[j]]).T).T
    return hr
def rot2hess(h):
        return rothess(rothess(h))
def rotgrad(g):
    b=r.apply(g)
    b[[2,3,4]]=b[[3,4,2]]
    return b 
def rot2grad(g):
        return rotgrad(rotgrad(g))

In [27]:
np.allclose(rothess(h0),h0)

True

In [28]:
np.allclose(rot2grad(afH3),afH2)

True

In [29]:
def reflectv(v,i,j):
    v1=ch4coords[5-i]
    v2=ch4coords[5-j]
    vs=v1+v2
    vp=np.cross(v1,v2)
    vo=np.cross(vs,vp)
    vo/=norm(vo)
    return v-(v.dot(vo))*2*vo

In [30]:
def reflgrad(g,i,j):
    if i==j:
        return g
    gr=np.zeros_like(g)
    for k in range(g.shape[0]):
        gr[k]=reflectv(g[k],i,j)
    gr[[5-i,5-j]]=gr[[5-j,5-i]]
    return gr

In [31]:
reflgrad(afH1,1,1)

array([[-0.02542, -0.01467, -0.01038],
       [-0.0125 , -0.00722, -0.011  ],
       [-0.0125 , -0.01278, -0.00314],
       [-0.01732, -0.00444, -0.00314],
       [ 0.06774,  0.03911,  0.02766]])

## First derivative 
$$ APDFT1(g)=  \sum_I\frac{\partial g}{\partial \lambda_I} =\sum_I F_I $$
all derivative are equal and can be computed analytically , $F$ is the alchemical force. 
## Second derivative 
$$ APDFT2(g)= \sum_I (\frac{\partial F_I}{\partial \lambda_I} 
+2 \frac{\partial }{\partial \lambda_I}\sum_{J\ne I} F_J )
$$

## Third derivative 
$$ APDFT3(g)= \sum_K \frac{\partial}{\partial \lambda_K}\sum_I (\frac{\partial F_I}{\partial \lambda_I} 
+2 \frac{\partial }{\partial \lambda_I}\sum_{J\ne I} F_J )
$$ 

In [32]:
Tm=FcM(fcs=[1,0,0,0,-1],atom=ch4str,unit="bohr",basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])})
Tmf = scf.RHF(Tm)
Tmf.scf(dm0=Tmf.init_guess_by_1e())

converged SCF energy = -56.2053208898248


-56.205320889824826

In [33]:
Tg=Tmf.Gradients().grad()
Th=Tmf.Hessian().hess()

--------------- RHF gradients ---------------
         x                y                z
0 C    -0.0379252172    -0.0218961344    -0.0154829051
1 H    -0.0052084464    -0.0030070979     0.0634324768
2 H    -0.0052084464     0.0588023467    -0.0239792781
3 H     0.0483201028    -0.0339118203    -0.0239792781
4 H     0.0000220072     0.0000127059     0.0000089844
----------------------------------------------


In [34]:
#p , m , pp, mm
dl=.1

In [35]:
CH4_p=FcM(fcs=[dl,0,0,0,-dl],atom=ch4str,unit="bohr",basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])})
mf_p = scf.RHF(CH4_p)
mf_p.scf(dm0=mf_p.init_guess_by_1e())
af_p1=aaff(mf_p,[1,0,0,0,-1])+alc_deriv_grad_nuc(CH4_p,[1,0,0,0,-1])
hp=mf_p.Hessian().hess()

converged SCF energy = -41.6026363391306


In [36]:
CH4_p2=FcM(fcs=[2*dl,0,0,0,-2*dl],atom=ch4str,unit="bohr",basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])})
mf_p2 = scf.RHF(CH4_p2)
mf_p2.scf(dm0=mf_p2.init_guess_by_1e())
af_p2=aaff(mf_p2,[1,0,0,0,-1])+alc_deriv_grad_nuc(CH4_p2,[1,0,0,0,-1])
hp2=mf_p2.Hessian().hess()

converged SCF energy = -43.0388427656976


In [37]:
CH4_m=FcM(fcs=[-dl,0,0,0,dl],atom=ch4str,unit="bohr",basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])})
mf_m = scf.RHF(CH4_m)
mf_m.scf(dm0=mf_m.init_guess_by_1e())
af_m1=aaff(mf_m,[1,0,0,0,-1])+alc_deriv_grad_nuc(CH4_m,[1,0,0,0,-1])
hm=mf_m.Hessian().hess()

converged SCF energy = -38.8771643174895


In [38]:
CH4_m2=FcM(fcs=[-2*dl,0,0,0,2*dl],atom=ch4str,unit="bohr",basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])})
mf_m2 = scf.RHF(CH4_m2)
mf_m2.scf(dm0=mf_m2.init_guess_by_1e())
af_m2=aaff(mf_m2,[1,0,0,0,-1])+alc_deriv_grad_nuc(CH4_m2,[1,0,0,0,-1])
hm2=mf_m2.Hessian().hess()

converged SCF energy = -37.589225638503


In [39]:
alc_deriv_grad_nuc(CH4_m,[1,0,0,0,-1]),alc_deriv_grad_nuc(CH4_m2,[1,0,0,0,-1])

(array([[ 1.13366,  0.65452,  0.46281],
        [ 0.04489,  0.02591, -0.16609],
        [ 0.04489, -0.14795,  0.0798 ],
        [-0.10569,  0.11285,  0.0798 ],
        [-1.11774, -0.64533, -0.45632]]),
 array([[ 1.09457,  0.63195,  0.44686],
        [ 0.04489,  0.02591, -0.16609],
        [ 0.04489, -0.14795,  0.0798 ],
        [-0.10569,  0.11285,  0.0798 ],
        [-1.07865, -0.62276, -0.44036]]))

In [40]:
af_1=afH1+afC
af_11=(af_m2/12-2/3*af_m1+2/3*af_p1-1/12*af_p2)/dl
af_111=(-1/12*af_m2 +4/3*af_m1-5/2*af_1+4/3*af_p1 -1/12*af_p2)/dl**2

In [41]:
adh_11=(hp-hm)/2/dl
adh_111=(hp+hm-2*h0)/dl**2

In [42]:
af_1111=(-1/2*af_m2+af_m1-af_p1+1/2*af_p2)/dl**3
af_11111=(af_m2-4*af_m1+6*af_1-4*af_p1+af_p2)/dl**4
adh_1111=(-1/2*hm2+hm-hp+1/2*hp2)/dl**3
adh_11111=(hm2-4*hm+6*h0-4*hp+hp2)/dl**4

In [43]:
ap2_g=af_1+af_11/2
ap2_h=h0+adh_11
ap3_g=af_1+af_11/2+af_111/6
ap3_h=h0+adh_11+adh_111/2
ap4_g=af_1+af_11/2+af_111/6+af_1111/24
ap4_h=h0+adh_11+adh_111/2+adh_1111/6second
ap5_g=af_1+af_11/2+af_111/6+af_1111/24+af_11111/120
ap5_h=h0+adh_11+adh_111/2+adh_1111/6+adh_11111/24

In [44]:
def nr_step(ic,g,h):
    ncoords=ic.shape[0]*ic.shape[1]
    p1cds=ic-np.linalg.solve((np.swapaxes(h,1,2)).reshape(ncoords,ncoords),g.reshape(-1)).reshape(ic.shape[0],ic.shape[1])
    p1cds-=p1cds[0]
    print (norm(p1cds[1]))
    if p1cds.shape[0]>2:
        print (anglecenter(p1cds),anglecenter(p1cds)*np.pi/180)
    return(p1cds)
def nr_step3(g,h):
    p1cds=ch4coords[:3]-np.linalg.solve((np.swapaxes(h,1,2)).reshape(9,9),g.reshape(-1)).reshape(3,3)
    p1cds-=p1cds[0]
    print (norm(p1cds[1]),anglecenter(p1cds),anglecenter(p1cds)*np.pi/180)
    return(p1cds)

In [45]:
p1cds=nr_step(ch4coords,af_1,h0)

1.8771028114069646
103.102 1.7994693653911935


In [46]:
norm(nr_step(ch4coords,af_1,ap2_h)[1])+mpa_pb(ch4coords,["N","H","H","H","H"],af_1,ap2_h)

1.8353590585595736
104.377 1.8217223133541214


1.9027895830284565

In [47]:
mpa_pb(ch4coords,["N","H","H","H","H"],Tg,Th,ghost=[4],solve_ic=True)

(array([2.04385, 2.04385, 2.04385, 2.04385, 1.91063, 1.91063, 1.91063,
        1.91063, 1.91063, 1.91063]),
 array([-0.21433, -0.21433, -0.21433,  0.82175, -2.84084, -2.84083,
        -2.84151, -2.84082, -2.84129, -2.84104]),
 array([ 1.82952,  1.82952,  1.82952,  2.8656 , -0.93021, -0.9302 ,
        -0.93087, -0.93019, -0.93066, -0.9304 ]),
 array([ 1.88992,  1.88991,  1.88991,  2.8656 , -0.40588, -0.40587,
        -0.35087, -0.40586, -0.35069, -0.35041]))

In [48]:
anglecenter(nr_step(ch4coords,af_1,h0))

1.8771028114069646
103.102 1.7994693653911935


103.102

In [49]:
mpa_pb(ch4coords,["N","H","H","H","H"],ap5_g,ap5_h,ghost=[4],solve_ic=True)

(array([2.04385, 2.04385, 2.04385, 2.04385, 1.91063, 1.91063, 1.91063,
        1.91063, 1.91063, 1.91063]),
 array([-0.21208, -0.21208, -0.21208,  0.07978, -0.0589 , -0.0589 ,
        -0.041  , -0.0589 , -0.04101, -0.04103]),
 array([1.83177, 1.83177, 1.83177, 2.12364, 1.85173, 1.85174, 1.86964,
        1.85174, 1.86962, 1.8696 ]),
 array([1.89164, 1.89165, 1.89165, 2.12364, 2.16243, 2.16243, 2.22797,
        2.16243, 2.22796, 2.22794]))

In [50]:
af_1m=af_1.copy()
ap2_gm=ap2_g.copy()
ap3_gm=ap3_g.copy()
ap4_gm=ap4_g.copy()
ap5_gm=ap5_g.copy()

In [51]:
af_11111/120

array([[-0.03513, -0.02028, -0.01434],
       [-0.00024, -0.00014, -0.0007 ],
       [-0.00024, -0.00071,  0.0001 ],
       [-0.00073,  0.00015,  0.0001 ],
       [ 0.03634,  0.02098,  0.01484]])

In [52]:
ap2_g,ap3_g,ap4_g,ap5_g,Tg

(array([[-0.11953, -0.06901, -0.0488 ],
        [-0.00548, -0.00316,  0.06576],
        [-0.00548,  0.06095, -0.0249 ],
        [ 0.05004, -0.03522, -0.0249 ],
        [ 0.08045,  0.04645,  0.03284]]),
 array([[-0.06049, -0.03493, -0.0247 ],
        [-0.00399, -0.0023 ,  0.0639 ],
        [-0.00399,  0.05948, -0.02347],
        [ 0.04951, -0.0332 , -0.02347],
        [ 0.01896,  0.01095,  0.00774]]),
 array([[-0.004  , -0.00231, -0.00163],
        [-0.00539, -0.00311,  0.06381],
        [-0.00539,  0.05912, -0.02421],
        [ 0.0485 , -0.03423, -0.02421],
        [-0.03372, -0.01947, -0.01377]]),
 array([[-0.03912, -0.02259, -0.01597],
        [-0.00563, -0.00325,  0.0631 ],
        [-0.00563,  0.05841, -0.0241 ],
        [ 0.04777, -0.03408, -0.0241 ],
        [ 0.00262,  0.00151,  0.00107]]),
 array([[-0.03793, -0.0219 , -0.01548],
        [-0.00521, -0.00301,  0.06343],
        [-0.00521,  0.0588 , -0.02398],
        [ 0.04832, -0.03391, -0.02398],
        [ 0.00002,  0.00001,  0.

In [53]:
af_1m[0]+=af_1[4]
af_1m[4]-=af_1[4]
ap2_gm[0]+=ap2_g[4]
ap2_gm[4]-=ap2_g[4]
ap3_gm[0]+=ap3_g[4]
ap3_gm[4]-=ap3_g[4]
ap4_gm[0]+=ap4_g[4]
ap4_gm[4]-=ap4_g[4]
ap5_gm[0]+=ap5_g[4]
ap5_gm[4]-=ap5_g[4]

In [54]:
print(norm(af_1m-Tg)),print(norm(ap2_gm-Tg)),print(norm(ap3_gm-Tg)),print(norm(ap4_gm-Tg)),print(norm(ap5_gm-Tg))

0.02546153217188889
0.00431670377131287
0.005106104162426655
0.0007911982465517387
0.0020191719939381693


(None, None, None, None, None)

In [55]:

print("1st order" ,mpa_pb(ch4coords[:4],["N","H","H","H"],af_1m[:4],ap2_h[:4,:4,:,:],solve_ic=True,ghost=[4])[3])
print("2nd order" ,mpa_pb(ch4coords[:4],["N","H","H","H"],ap2_gm[:4],ap3_h[:4,:4,:,:],solve_ic=True,ghost=[4])[3])
print("3rd order" ,mpa_pb(ch4coords[:4],["N","H","H","H"],ap3_gm[:4],ap4_h[:4,:4,:,:],solve_ic=True,ghost=[4])[3])
print("4th order" ,mpa_pb(ch4coords[:4],["N","H","H","H"],ap4_gm[:4],ap5_h[:4,:4,:,:],solve_ic=True,ghost=[4])[3])

print("with target grad" ,mpa_pb(ch4coords[:4],["N","H","H","H"],Tg[:4],Th[:4,:4,:,:],solve_ic=True,ghost=[4])[3])

1st order [1.89735 1.89735 1.89735 1.84036 1.84036 1.84036]
2nd order [1.8829  1.8829  1.8829  1.86955 1.86955 1.86955]
3rd order [1.86453 1.86453 1.86453 1.95352 1.95352 1.95352]
4th order [1.89015 1.89015 1.89016 1.88495 1.88495 1.88495]
with target grad [1.88991 1.88991 1.88992 1.88554 1.88554 1.88554]


In [56]:
print(mpa_pb(ch4coords[:4],["N","H","H","H"],ap2_gm[:4],ap3_h[:4,:4,:,:],solve_ic=True,ghost=[4])[3])
print(mpa_pb(ch4coords[:4],["N","H","H","H"],ap2_g[:4],ap3_h[:4,:4,:,:],solve_ic=True,ghost=[4])[3])

[1.8829  1.8829  1.8829  1.86955 1.86955 1.86955]
[1.99274 1.99276 1.99282 1.12369 1.12349 1.12339]


In [57]:
nr_step(ch4coords[:4],ap2_gm[:4],ap3_h[:4,:4,:,:])
nr_step(ch4coords,ap2_gm,ap3_h)
nr_step(ch4coords,af_1,h0)
nr_step(ch4coords,Tg,Th)

1.8196303013134536
109.171 1.9053933976947295
1.8196303079132299
109.171 1.9053933976947295
1.8771028114069646
103.102 1.7994693653911935
1.830545202777996
109.12 1.9045032797762123


array([[ 0.     ,  0.     ,  0.     ],
       [ 0.00965,  0.00557,  1.83051],
       [ 0.00965,  1.72768, -0.60492],
       [ 1.50104, -0.85549, -0.60492],
       [-2.27065, -1.31096, -0.92699]])

In [58]:
minimum

(1.8856061076490165, 1.888324077610225)

In [59]:
print(mpa_pb(ch4coords[:4],["N","H","H","H"],af_1m[:4],h0[:4,:4,:,:],solve_ic=True,ghost=[4])[3])
print(mpa_pb(ch4coords[:4],["N","H","H","H"],af_1m[:4],ap2_h[:4,:4,:,:],solve_ic=True,ghost=[4])[3])
print(mpa_pb(ch4coords[:4],["N","H","H","H"],ap2_gm[:4],ap3_h[:4,:4,:,:],solve_ic=True,ghost=[4])[3])
print(mpa_pb(ch4coords[:4],["N","H","H","H"],ap3_gm[:4],ap4_h[:4,:4,:,:],solve_ic=True,ghost=[4])[3])
print(mpa_pb(ch4coords[:4],["N","H","H","H"],ap4_gm[:4],ap5_h[:4,:4,:,:],solve_ic=True,ghost=[4])[3])
print(mpa_pb(ch4coords[:4],["N","H","H","H"],Tg[:4],Th[:4,:4,:,:],solve_ic=True,ghost=[4])[3])

[1.91493 1.91493 1.91493 1.83769 1.8377  1.8377 ]
[1.89735 1.89735 1.89735 1.84036 1.84036 1.84036]
[1.8829  1.8829  1.8829  1.86955 1.86955 1.86955]
[1.86453 1.86453 1.86453 1.95352 1.95352 1.95352]
[1.89015 1.89015 1.89016 1.88495 1.88495 1.88495]
[1.88991 1.88991 1.88992 1.88554 1.88554 1.88554]


In [60]:
print(mpa_pb(ch4coords[:4],["N","H","H","H"],ap2_gm[:4],ap3_h[:4,:4,:,:],solve_ic=True,ghost=[4]))
print(mpa_pb(ch4coords[:4],["N","H","H","H"],ap2_g[:4],ap3_h[:4,:4,:,:],solve_ic=True,ghost=[4]))

(array([2.04385, 2.04385, 2.04385, 1.91063, 1.91063, 1.91063]), array([-0.22924, -0.22924, -0.22924,  0.01867,  0.01867,  0.01867]), array([1.81461, 1.81461, 1.81462, 1.9293 , 1.9293 , 1.9293 ]), array([1.8829 , 1.8829 , 1.8829 , 1.86955, 1.86955, 1.86955]))
(array([2.04385, 2.04385, 2.04385, 1.91063, 1.91063, 1.91063]), array([-0.13855, -0.13853, -0.13848, -0.86039, -0.8606 , -0.86072]), array([1.9053 , 1.90532, 1.90538, 1.05024, 1.05003, 1.04992]), array([1.99274, 1.99276, 1.99282, 1.12369, 1.12349, 1.12339]))


In [61]:
ap2_gm[:],ap2_g[:],ap3_g,ap4_g,ap5_g,Tg

(array([[-0.03909, -0.02257, -0.01596],
        [-0.00548, -0.00316,  0.06576],
        [-0.00548,  0.06095, -0.0249 ],
        [ 0.05004, -0.03522, -0.0249 ],
        [ 0.     ,  0.     ,  0.     ]]),
 array([[-0.11953, -0.06901, -0.0488 ],
        [-0.00548, -0.00316,  0.06576],
        [-0.00548,  0.06095, -0.0249 ],
        [ 0.05004, -0.03522, -0.0249 ],
        [ 0.08045,  0.04645,  0.03284]]),
 array([[-0.06049, -0.03493, -0.0247 ],
        [-0.00399, -0.0023 ,  0.0639 ],
        [-0.00399,  0.05948, -0.02347],
        [ 0.04951, -0.0332 , -0.02347],
        [ 0.01896,  0.01095,  0.00774]]),
 array([[-0.004  , -0.00231, -0.00163],
        [-0.00539, -0.00311,  0.06381],
        [-0.00539,  0.05912, -0.02421],
        [ 0.0485 , -0.03423, -0.02421],
        [-0.03372, -0.01947, -0.01377]]),
 array([[-0.03912, -0.02259, -0.01597],
        [-0.00563, -0.00325,  0.0631 ],
        [-0.00563,  0.05841, -0.0241 ],
        [ 0.04777, -0.03408, -0.0241 ],
        [ 0.00262,  0.00151,  0.

In [62]:
print(mpa_pb(ch4coords[:4],["N","H","H","H"],af_1m[:4],h0[:4,:4,:,:],solve_ic=True,ghost=[4])[3])

[1.91493 1.91493 1.91493 1.83769 1.8377  1.8377 ]


In [63]:
af_h2o=2*afC+afH1+afH2
af_h2o[0]=af_h2o[0]+af_h2o[4]+af_h2o[3]
af_h2o[4]-=af_h2o[4]
af_h2o[3]-=af_h2o[3]
print(mpa_pb(ch4coords[:3],["O","H","H"],af_h2o[:3],h0[:3,:3,:,:],solve_ic=True))

(array([2.04385, 2.04385, 1.91063]), array([-0.3542 , -0.3542 , -0.33133]), array([1.68965, 1.68965, 1.5793 ]), array([1.86136, 1.86136, 1.5989 ]))


In [64]:
af_2=rotgrad(af_1)
af_22=rotgrad(af_11)
af_222=rotgrad(af_111)

In [65]:
af_12=(aaff(mf_p,[1,0,0,-1,0])+alc_deriv_grad_nuc(CH4_p,[1,0,0,-1,0])-aaff(mf_m,[1,0,0,-1,0])-alc_deriv_grad_nuc(CH4_m,[1,0,0,-1,0]))/2/dl

In [66]:
af_h2o=af_1+af_2+af_11/2+af_22/2+af_12
af_h2o[0]=af_h2o[0]+af_h2o[4]+af_h2o[3]
af_h2o[4]-=af_h2o[4]
af_h2o[3]-=af_h2o[3]
print(mpa_pb(ch4coords[:3],["O","H","H"],af_h2o[:3],h0[:3,:3,:,:],solve_ic=True))

(array([2.04385, 2.04385, 1.91063]), array([-0.30272, -0.30271, -0.27125]), array([1.74114, 1.74114, 1.63939]), array([1.86966, 1.86966, 1.65165]))


In [69]:
nr_step3(af_h2o[:3],h0[:3,:3,:,:])

1.7397814314167623 100.758 1.7585588477244465


array([[ 0.     ,  0.     ,  0.     ],
       [-0.00047,  0.13214,  1.73476],
       [-0.00047,  1.67961, -0.45362]])

In [70]:
CH4_pp=FcM(fcs=[2*dl,0,0,-dl,-dl],atom=ch4str,unit="bohr",basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])})
mf_pp = scf.RHF(CH4_pp)
mf_pp.scf(dm0=mf_pp.init_guess_by_1e())
af_pp=aaff(mf_pp,[2,0,0,-1,-1])+alc_deriv_grad_nuc(CH4_pp,[1,0,0,-1,-1])
hpp=mf_pp.Hessian().hess()

converged SCF energy = -43.0255810209441


In [71]:
CH4_mm=FcM(fcs=[-2*dl,0,0,dl,dl],atom=ch4str,unit="bohr",basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])})
mf_mm = scf.RHF(CH4_mm)
mf_mm.scf(dm0=mf_mm.init_guess_by_1e())
af_mm=aaff(mf_pp,[2,0,0,-1,-1])+alc_deriv_grad_nuc(CH4_mm,[2,0,0,-1,-1])
hmm=mf_mm.Hessian().hess()

converged SCF energy = -37.5739249489244


In [72]:
aph_h20=h0+(hpp-hmm)/2/dl+(hpp-2*h0+hmm)/dl**2/2

In [73]:
adh_12=hpp+2*h0+hmm-hm-rothess(hm)-hp-rothess(hp)

In [74]:
nr_step3(af_h2o[:3],aph_h20[:3,:3,:,:])
nr_step(ch4coords,af_h2o,aph_h20)
nr_step(ch4coords,af_1+af_2+af_11/2+af_22/2+af_12,aph_h20)
print(mpa_pb(ch4coords[:3],["O","H","H"],af_h2o[:3],aph_h20[:3,:3,:,:],solve_ic=True))

1.4346292486304917 103.789 1.8114597773523946
1.4427321051513435
103.44 1.8053685782629345
1.37509711718096
111.218 1.9411202874830533
(array([2.04385, 2.04385, 1.91063]), array([-0.74952, -0.74953,  0.32804]), array([1.29434, 1.29433, 2.23867]), array([1.69966, 1.69965, 1.87201]))


In [75]:
af_hf=3*afC+afH1+afH2+afH3
af_hf[0]=af_hf[0]+af_hf[4]+af_hf[3]+af_hf[2]
af_hf[4]-=af_hf[4]
af_hf[3]-=af_hf[3]
af_hf[2]-=af_hf[2]
print(mpa_pb(ch4coords[:2],["F","H"],af_hf[:2],h0[:2,:2,:,:],solve_ic=True))

(array([2.04385]), array([-0.49554]), array([1.54831]), array([1.8477]))


In [76]:
print(mpa_pb(ch4coords[:2],["F","H"],(3*afC+afH1+afH2+afH3)[:2],h0[:2,:2,:,:],solve_ic=True))

(array([2.04385]), array([-0.28562]), array([1.75823]), array([1.86984]))


In [77]:
adh_22=rothess(adh_11)
adh_33=rothess(adh_22)

In [78]:
nr_step(ch4coords,3*afC+afH1+afH2+afH3,h0)
nr_step(ch4coords[:2],af_hf[:2],(h0+adh_11+adh_22+adh_33)[:2,:2,:,:])

1.5148044848371924
103.079 1.7990679396632347
0.8187752076394275


array([[ 0.     ,  0.     ,  0.     ],
       [ 0.     , -0.     ,  0.81878]])

In [79]:
print(mpa_pb(ch4coords[:2],["F","H"],af_hf[:2],(h0+adh_11+adh_22+adh_33)[:2,:2,:,:],solve_ic=True))
print(mpa_pb(ch4coords[:2],["F","H"],(3*afC+afH1+afH2+afH3)[:2],(h0+adh_11+adh_22+adh_33)[:2,:2,:,:],solve_ic=True))

(array([2.04385]), array([-1.07866]), array([0.96519]), array([1.7787]))
(array([2.04385]), array([-0.62173]), array([1.42213]), array([1.75661]))


In [80]:
fhoptg

array([[0.     , 0.     , 0.     ],
       [0.     , 0.     , 1.69574]])